In [11]:
#First Step @ second layer
import string
import requests
from bs4 import BeautifulSoup

filename_list={}
fl=1

cookie = 'ASP.NET_SessionId=y13rhiyxsgjugze1tko01lvf; lawbank=90ba58ddea3041daae26ad8c56b576f2-39; _ga=GA1.3.1224627297.1434421447; _gat=1; JubFrm-pagebox=%5EcourtFullName%3DILDV%60%5Ekw%3D%E4%BA%A4%E9%80%9A%E4%BA%8B%E6%95%85%5Ejcatagory%3D0%5Eissimple%3D-1%5Ejt%3D%E6%90%8D%E5%AE%B3%E8%B3%A0%E5%84%9F; x=j=csoRwOAahQP30doG2B84Jl6rT2X696guyiYjzyfERzlUq0jgk9RwqJTmSuWR4UjIFn3eMQ8OQeamsVBMIcLBwB/qRb6KvfqhvRN1QMUCZCgGJ9TCjrxJ/6F3RWp/jnwfGfKzFqpL4EiGiR3tiiwKr/coRWM8KpY+nP9QguBrCYKwIuKNUBMsP2L91/2sWT5M0TtoS5/hYYwmPNBL9VZh6w1w7I4hNKVsOvjsXMkUaHP3bhMkAaY6IxVrP6AknAs/zeyjI1EcoGol9cUJXJft7+fRlTyDBCaPR9fnl1pI2Siv2QWD/bcTOCy0Nv2MRXlJDFzLytstoApjPV9xeu4hmw=='

#catch y=p= whether exist
if cookie.find('y=p=') != -1:
    pos_yp = cookie.split('y=p=')
    cookie = pos_yp[0]+string.join(pos_yp[1].split(';')[1:],';')

uml = "http://fyjud.lawbank.com.tw/"
headers = {'Cookie' : cookie}
res = requests.get('http://fyjud.lawbank.com.tw/listcontent5.aspx',\
                   headers = headers)
soup = BeautifulSoup(res.text)

#抓location ----- place 
#<eg台北>
for h4 in soup.select('h4'):
    allname = str(h4).decode('utf-8')
    pos = allname.find('地方法院'.decode('utf-8'))
    place = allname[pos-2:pos].encode('utf-8')

#catch total_pages
state = soup.select('.page > tr > td')[0].text.replace(' ','')
pos_start = state.find(u'共')
pos_end = state.find(u'筆')
total_pages = int(state[pos_start+1:pos_end])

#進到第三層
data = {'__EVENTTARGET':'lnkbtnOhmy1', \
        '__EVENTARGUMENT':'', \
        '__VIEWSTATE':'', \
        '__VIEWSTATEGENERATOR':'', \
        '__EVENTVALIDATION':'', \
        'FDLink':''
       }
a = soup.select('table > tbody > tr > td > a')[0]
link = a['href']
cookieId = a['onclick'].split('\'')[1]
headers_3rd = {'Cookie': '%s;y=p=%s'%(cookie, cookieId)}
res_3rd = requests.get('%s'%(uml+link), headers = headers_3rd)
soup_3rd = BeautifulSoup(res_3rd.text)

#each file name ----- save_name
#eg. 士林_103_士簡_867
import string
save= soup_3rd.select('#FDLink')[0]['value'].encode('utf-8').split(',')
save_name = string.join(save[2:5],'_')
filename_list[fl]=(place+'_'+save_name+'.txt').decode('utf-8')
fl+=1

#save first soup_3rd by get
fid=open('project\%s'%(filename_list[fl-1]),'w')
pre_len = len(soup_3rd.select('pre'))
for i in range(0,pre_len):
    content = soup_3rd.select('pre')[i].text.encode('utf-8')
    fid.write(content+'\n')
fid.close()
    
        #save_name
import time
try:
    for i in range(1, total_pages):
        data['__VIEWSTATE'] = soup_3rd.select('#__VIEWSTATE')[0]['value']
        data['__EVENTVALIDATION'] = soup_3rd.select('#__EVENTVALIDATION')[0]['value']
        data['__VIEWSTATEGENERATOR'] = soup_3rd.select('#__VIEWSTATEGENERATOR')[0]['value']

        res_post = requests.post('http://fyjud.lawbank.com.tw/content3.aspx?p=lJE0csYFgzBIuZknAS%2f9cG4vHeoH7hup9KUMRu4jjlc%3d',\
                           headers = headers_3rd,\
                           data = data)
        soup_3rd = BeautifulSoup(res_post.text)

        save= soup_3rd.select('#FDLink')[0]['value'].encode('utf-8').split(',')
        save_name = string.join(save[2:5],'_')
        save_dir = save[1]
        filename_list[fl]=(place+'_'+save_name+'.txt').decode('utf-8')
        fl+=1
    
        fid=open('project\%s'%(filename_list[fl-1]),'w')

        pre_len = len(soup_3rd.select('pre'))
        for pl in range(0,pre_len):
            content = soup_3rd.select('pre')[pl].text.encode('utf-8')
            fid.write(content+'\n')
        fid.close()
        if i%100==0:
            print i,'-ok'
        time.sleep(1)

finally:
#make an index by filename_list to a file
#open() code must be 'unicode'
#but write() code is 'utf-8'
    total = len(filename_list)
    save_filename_list = ('project\%s.txt'\
                    %place).decode('utf-8')
    fn=open(save_filename_list,'w')
    for i in range(1,total+1):
        fn.write(str(i)+'-'+filename_list[i].encode('utf-8')+'\n')
    fn.close()
    if total==total_pages:
        print 'you did caught everything, there are %d files!'%total
    else:
        print 'FAIL!!!, there should be %d files!'%total
        print 'but u only got %d files...'%total_pages

100 -ok
200 -ok
300 -ok
400 -ok
500 -ok
FAIL!!!, there should be 510 files!
but u only got 696 files...


ConnectionError: ('Connection aborted.', error(10060, '\xb3s\xbdu\xb9\xc1\xb8\xd5\xa5\xa2\xb1\xd1\xa1A\xa6]\xac\xb0\xb3s\xbdu\xb9\xef\xb6H\xa6\xb3\xa4@\xacq\xae\xc9\xb6\xa1\xa8\xc3\xa5\xbc\xa5\xbf\xbdT\xa6^\xc0\xb3\xa1A\xa9\xce\xacO\xb3s\xbdu\xab\xd8\xa5\xdf\xa5\xa2\xb1\xd1\xa1A\xa6]\xac\xb0\xb3s\xbdu\xaa\xba\xa5D\xbe\xf7\xb5L\xaak\xa6^\xc0\xb3\xa1C'))